# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 50)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 50:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(word_list):
    """
    Create lookup tables for vocabulary
    :param wordlist: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    
    # Scan each line of the word list for new words and add them to the counter dictionary
    word_counts = Counter(word_list)
    
    # sort the words in word_counts dictionary from the most to the least frequent in text occurence
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
        
    # Create a dictionary of words from the sorted vocabulary mapped to index positions 
    # and another dictionary of index positions mapped to the words of the vocabulary
    int_to_vocab = {idx: word for idx, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: idx for idx, word in int_to_vocab.items()}
    
    # return tuple of dictionaries
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token_dict = {'.': '||Period||',
                  ',': '||Comma||',
                  '"': '||Quotation_Mark||',
                  ';': '||Semicolon||',
                  '!': '||Exclamation_Mark||',
                  '?': '||Question_Mark||',
                  '(': '||Left_Parentheses||',
                  ')': '||Right_Parentheses||',
                  '-': '||Dash||',
                  '\n': '||Return||'
                 }
    return token_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to look at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

# Analyse the vocabulary

In [7]:
print('Total number of words in training text:  ', len(int_text))
print('Total number of words in the vocabulary: ', len(vocab_to_int), '(incl. token for padding)')

Total number of words in training text:   892110
Total number of words in the vocabulary:  21388 (incl. token for padding)


In [8]:
# Get a token counter dictionary with the frequency of the tokenized words' appearing in the text
token_counts = Counter(int_text)
# Sort word counter dictionary by the frequency of appearance in the text
sorted_token_counts = {key: value for key, value in sorted(token_counts.items(), key=lambda item: item[1], reverse=True)}
# Print token counter dictionary
print('Sorted tokenized vocabulary:\n{}'.format(sorted_token_counts))

Sorted tokenized vocabulary:
{0: 109232, 1: 73969, 2: 39304, 3: 20940, 4: 19935, 5: 18466, 6: 17933, 7: 14805, 8: 12910, 9: 12832, 10: 10697, 11: 10670, 12: 10659, 13: 9729, 14: 8005, 15: 7794, 16: 6682, 17: 6523, 18: 6238, 19: 5936, 20: 5755, 21: 5504, 22: 5176, 23: 4915, 24: 4812, 25: 4712, 26: 4442, 27: 4204, 28: 4132, 29: 4125, 30: 3843, 31: 3789, 32: 3783, 33: 3765, 34: 3440, 35: 3407, 36: 3377, 37: 3371, 38: 3370, 39: 3356, 40: 3277, 41: 3142, 42: 3134, 43: 2887, 44: 2880, 45: 2873, 46: 2819, 47: 2728, 48: 2718, 49: 2680, 50: 2664, 51: 2601, 52: 2548, 53: 2526, 54: 2451, 55: 2439, 56: 2421, 57: 2370, 58: 2234, 59: 2218, 60: 2116, 61: 2099, 62: 2080, 63: 2018, 64: 2010, 65: 1982, 66: 1969, 67: 1967, 68: 1919, 69: 1868, 70: 1855, 71: 1805, 72: 1788, 73: 1770, 74: 1769, 75: 1767, 76: 1756, 77: 1675, 78: 1605, 79: 1562, 80: 1553, 81: 1536, 82: 1517, 83: 1476, 84: 1461, 85: 1428, 86: 1418, 87: 1403, 88: 1359, 89: 1342, 90: 1318, 91: 1285, 92: 1267, 93: 1251, 94: 1241, 95: 1204, 96: 11

In [9]:
# Translate tokens in to words (not sorted by frequency any more)
word_counts = {int_to_vocab[idx]: count for idx, count in sorted_token_counts.items()}

# Sort word counter dictionary by the frequency of appearance in the text
sorted_word_counts = {key: value for key, value in sorted(word_counts.items(), key=lambda item: item[1], reverse=True)}
# Print word counter dictionary
print('Sorted vocabulary:\n{}'.format(sorted_word_counts))

Sorted vocabulary:
{'||return||': 109232, '||period||': 73969, '||comma||': 39304, '||question_mark||': 20940, 'you': 19935, 'i': 18466, 'the': 17933, 'jerry:': 14805, 'to': 12910, 'a': 12832, '||exclamation_mark||': 10697, '||left_parentheses||': 10670, '||right_parentheses||': 10659, 'george:': 9729, 'elaine:': 8005, 'it': 7794, 'kramer:': 6682, 'and': 6523, 'what': 6238, 'that': 5936, 'of': 5755, 'in': 5504, 'is': 5176, '||dash||': 4915, 'this': 4812, 'me': 4712, 'no': 4442, 'know': 4204, 'on': 4132, 'oh': 4125, 'yeah': 3843, "i'm": 3789, 'well': 3783, 'my': 3765, 'do': 3440, 'he': 3407, "it's": 3377, 'so': 3371, 'for': 3370, 'have': 3356, 'with': 3277, 'was': 3142, "don't": 3134, 'jerry': 2887, 'just': 2880, 'not': 2873, 'like': 2819, 'out': 2728, 'are': 2718, 'up': 2680, 'all': 2664, 'get': 2601, 'your': 2548, '||quotation_mark||': 2526, 'be': 2451, 'we': 2439, 'hey': 2421, 'right': 2370, 'about': 2234, 'at': 2218, 'george': 2116, 'got': 2099, 'here': 2080, 'her': 2018, 'go': 2010

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [11]:
from torch.utils.data import TensorDataset, DataLoader

def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The list of word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    
    # possible number of full batches with respect to the given batch size and sequence length per batch
    n_batches = (len(words)-sequence_length)//batch_size
    
    # convert list of word ids into a numpy array
    words_arr = np.array(words, dtype=np.int64)
    
    # initialize feature and target tensors considering only full batches of sequences of word ids
    # discarding the rest at the end of the list of word ids
    feature_tensors = np.zeros((n_batches*batch_size, sequence_length), dtype=np.int64)
    target_tensors = np.zeros((n_batches*batch_size, ), dtype=np.int64)
    # fill in feature tensors with a sequence of word ids
    for c in range(0, sequence_length):
        feature_tensors[0:n_batches*batch_size, c] = words_arr[c:c+n_batches*batch_size].T
    # fill in target tensors with the word ids following up directly after the corresponding feature sequence
    target_tensors[0:n_batches*batch_size] = words_arr[sequence_length:sequence_length+n_batches*batch_size]
    
    # create tensor data set consisting of a series of feature and target tensors
    # data = TensorDataset(torch.from_numpy(np.array(feature_tensors)), torch.from_numpy(np.array(target_tensors)))
    data = TensorDataset(torch.from_numpy(feature_tensors), torch.from_numpy(target_tensors))
    
    # create data loader considering data shuffling
    data_loader = torch.utils.data.DataLoader(data, shuffle=True, batch_size=batch_size)
    
    # return a dataloader
    return data_loader


# there is no test for this function, but you are encouraged to create
# print statements and tests of your own
data_loader = batch_data(int_text, sequence_length=10, batch_size=5)

### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [12]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[38, 39, 40, 41, 42],
        [24, 25, 26, 27, 28],
        [ 6,  7,  8,  9, 10],
        [31, 32, 33, 34, 35],
        [ 9, 10, 11, 12, 13],
        [18, 19, 20, 21, 22],
        [26, 27, 28, 29, 30],
        [33, 34, 35, 36, 37],
        [10, 11, 12, 13, 14],
        [20, 21, 22, 23, 24]])

torch.Size([10])
tensor([43, 29, 11, 36, 14, 23, 31, 38, 15, 25])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [13]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden LSTM/GRU layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        ## set class variables
        
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        ## define model layers
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM/GRU layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        
        # Dropout layer between last LSTM/GRU layer and fully connected linear layer
        self.dropout = nn.Dropout(p=0.3)
        
        # Final fully-connected linear layer (classifier)
        self.fc = nn.Linear(hidden_dim, output_size)
        
        # as CrossEntropyLoss is used as optimization criterion there is no further softmax layer
        # the scorses of the final fully-connected linear layer are output directly
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function
        
        # get batch size from neural network input
        batch_size = nn_input.size(0)
        
        # transform input into long tensor (int64)
        nn_input = nn_input.long()
        
        # pass input through embedding layer
        embeddings = self.embedding(nn_input)
        
        # pass embeddings and previous hidden state trough lstm layer to get lstm output and next hidden state
        lstm_output, hidden = self.lstm(embeddings, hidden)
        
        # stack up lstm outputs as contiguous tensor before passing it to the fully-connected linear layer
        lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)
        
        # regularize the stacked lstm outputs of the previous time steps using dropout and pass them on
        # through the fully-connected classifier layer
        scores = self.fc(self.dropout(lstm_output))     
        
        # reshape the output scores into (batch_size, seq_length, output_size)
        scores = scores.view(batch_size, -1, self.output_size)
        
        # get only the last batch of predicted word id scores (ignoring the previous lstm steps)
        scores = scores[:, -1]
            
        # return one batch of output word id scores and the hidden state
        return scores, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        
        # initialize hidden state with zero weights, and move to GPU if available
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim, both initialized to zero, 
        # for hidden state and cell state of the LSTM layer
        
        # get next set of weights by iterating to the next data set in self.parameters
        weight = next(self.parameters()).data
        
        # create a tuple of two new tensors with sizes n_layers x batch_size x hidden_dim initialized to zero
        # for the hidden state and the cell state of the LSTM layer, and move them to the device
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        # return a tuple of two tensors of size (n_layers, batch_size, hidden_dim) for both hidden state 
        # and cell state of the LSTM layer and return them in the variable hidden
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [14]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    # hard-coded gradient clipping threshold
    grad_clip_threshold = 5
    
    # move data to GPU, if available
    if (train_on_gpu):
        inp, target = inp.cuda(), target.cuda() # move features and targets to gpu
        
    ## perform backpropagation and optimization
    
    # Create new variables for the hidden state, otherwise we'd backprop through the entire training history
    hidden = tuple([each.data for each in hidden])
    
    # zero accumulated gradients
    rnn.zero_grad()
    
    # get the predicted output scores and the next predicted hidden state from the rnn model
    output_scores, hidden = rnn(inp, hidden)

    # calculate the training batch loss and perform backprop
    loss = criterion(output_scores.squeeze(), target)
    
    # calculate the backward gradients for backpropagation
    loss.backward()
    
    # perform gradient clipping to prevent gradients from exploding
    # (clip_grad_norm`helps prevent the exploding gradient problem in RNNs / LSTMs)
    nn.utils.clip_grad_norm_(rnn.parameters(), grad_clip_threshold)
    
    # perform a backpropagation step
    optimizer.step()
    
    # get average batch loss
    avg_batch_loss = loss.item()
    
    # return the loss over a batch and the hidden state produced by our model
    return avg_batch_loss, hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [16]:
## Data parameters
#
# Sequence Length
sequence_length = 8  # sequence_length >= 5 (truncated average number of works per line in the training text)
# Batch Size
batch_size = 128

# Initialize data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [17]:
## Training parameters
#
# Number of Epochs
num_epochs = 50 # >= 10 required to reqch target loss of < 3.5
# Learning Rate
learning_rate = 0.001 # try 0.01, 0.001 (works best here), 0.0001

## Model parameters
#
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 512 # try 128, 256, 512 (slow), 1024 (too slow) => recommendation: 100...500 (e. g. https://adventuresinmachinelearning.com/keras-lstm-tutorial/)
# Hidden Dimension
hidden_dim = 512 # should match up with embedding_dim
# Number of RNN Layers
n_layers = 2 # try 2, 3

# Show stats for every n number of batches
show_every_n_batches = 1000

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [18]:
import time

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
tic = time.clock()
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)
toc = time.clock()
print('Processing time for training: ', toc - tic)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 50 epoch(s)...
Epoch:    1/50    Loss: 5.001144201755523

Epoch:    1/50    Loss: 4.499081241369248

Epoch:    1/50    Loss: 4.383104060649872

Epoch:    1/50    Loss: 4.291987797021866

Epoch:    1/50    Loss: 4.242453042268753

Epoch:    1/50    Loss: 4.18841120839119

Epoch:    2/50    Loss: 4.097142379982572

Epoch:    2/50    Loss: 4.013258145570755

Epoch:    2/50    Loss: 3.9983497910499572

Epoch:    2/50    Loss: 4.012476859331131

Epoch:    2/50    Loss: 4.003788537979126

Epoch:    2/50    Loss: 3.9908232243061064

Epoch:    3/50    Loss: 3.9205521486203887

Epoch:    3/50    Loss: 3.868745114326477

Epoch:    3/50    Loss: 3.8652566368579864

Epoch:    3/50    Loss: 3.8746536021232605

Epoch:    3/50    Loss: 3.8677168579101564

Epoch:    3/50    Loss: 3.872125678539276

Epoch:    4/50    Loss: 3.80554006631269

Epoch:    4/50    Loss: 3.7604932179450987

Epoch:    4/50    Loss: 3.7723721725940704

Epoch:    4/50    Loss: 3.7915618340969086

Epoch:    4/50    L

Epoch:   32/50    Loss: 3.319429590702057

Epoch:   32/50    Loss: 3.3213689811229705

Epoch:   32/50    Loss: 3.347652937889099

Epoch:   32/50    Loss: 3.369677860736847

Epoch:   33/50    Loss: 3.322583400761191

Epoch:   33/50    Loss: 3.292710266828537

Epoch:   33/50    Loss: 3.3171290910243987

Epoch:   33/50    Loss: 3.3252255532741546

Epoch:   33/50    Loss: 3.3378596007823944

Epoch:   33/50    Loss: 3.3574669020175936

Epoch:   34/50    Loss: 3.323013180703182

Epoch:   34/50    Loss: 3.2827984204292298

Epoch:   34/50    Loss: 3.296329185009003

Epoch:   34/50    Loss: 3.3293037259578706

Epoch:   34/50    Loss: 3.3256962010860445

Epoch:   34/50    Loss: 3.3445224609375

Epoch:   35/50    Loss: 3.3208406327520303

Epoch:   35/50    Loss: 3.272661211490631

Epoch:   35/50    Loss: 3.2991697890758513

Epoch:   35/50    Loss: 3.311001180410385

Epoch:   35/50    Loss: 3.33086923122406

Epoch:   35/50    Loss: 3.3768276290893553

Epoch:   36/50    Loss: 3.305902636311513

Epo

/home/andreas/anaconda3/envs/deep-learning/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** (Write answer, here)

- I have tried different learning rates of 0.01 to 0.0001 with the same setting of the other hyperparameters. A learning rate of 0.001 seems to work best for this problem yielding the quickest descent of the loss - at least for the first 3...5 epochs.
- As the average number of words per line in the training text is about 5.5 words, the sequence_length should be at least >= 5 words. This way a sequence would cover typical senence block arragements like this: "The cat sat on the ... (e. g. mat)" => I chose sequence_length = 8 as this seems to fit better with typical sentences given by the Seinfield manuscript like "jerry: i thought i told you about it, yes, ..." also compromise punctuation between words
- In our Udacity course Word2Vec example we have used an embedding dimension of 300 to embed a vocabulary of about 63641 unique words. The author of another source on LSTMs (https://adventuresinmachinelearning.com/keras-lstm-tutorial/) recommends embedding layers of similar size in the range 100...500. After several tests I have chosen 512 at the upper end of the recommended scale. 256 would also work. For a larger size of e. g. 1024 training takes too long for my liking.
- The author of https://adventuresinmachinelearning.com/keras-lstm-tutorial/ recommends to match up embedding and hidden dimension with one another. This seems to work well in my case, too. So I have finally chosen the same size of 512 for hidden_dim.
- The depth of the LSTM stack determines the learning potential. In most examples that have been discussed in the course 2 - 3 LSTM layers seem to work well for such problems. I have tried 2 and 3 with little difference. So I have chosen 2 for the sake of training time reduction. 

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    # set rnn model to evaluation mode
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index from the top_k word indeces with some element of randomness
        # using the probabilities of the top_k word indeces prediected by the rnn
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)
        
        # if(train_on_gpu):
        #    current_seq = current_seq.cpu() # move to cpu
        # Move current sequence to cpu
        if train_on_gpu:
            current_seq = current_seq.cpu()
        # the generated word becomes part of the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [21]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

jerry: oh, yeah. i got the pineapple. i got it.

jerry: i don't think i can be a vegetable lasagna.(george laughs)

kramer: well.. i know what i mean. i have a lot of friends.

elaine:(pause) i don't know, i think i'm gonna get some new free of you.

jerry: what about the whole planet about the library?

kramer: no.

newman: i know, i know.....

jerry:(to elaine) what about the dream.(to jerry) i know what this means is?

george: well i think i can get a chance to get out.

george: oh, no no, no..(george leaves.)

elaine: oh, no, no. i don't. i don't like it.

elaine: oh, i think that's a good sign.

jerry: oh, i don't want to see that.

elaine:(to kramer) hey, how are ya?

george: i don't know.

kramer: i got a date with paba on the street.(to jerry) so what do i do, i don't want to get a job.

elaine:(to george) hey, you want me to take a civil service test.

jerry: yeah.

george:(still laughing) what?

george: oh, yeah, sure, i was thinking, but i just want a racing car.

elaine:(to

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [22]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.